In [3]:
import sys
# This ensures you install it to the EXACT python path Jupyter is currently using
!{sys.executable} -m pip install albumentations opencv-python matplotlib

   ---------------------------------------- 0.0/40.1 MB ? eta -:--:--
    --------------------------------------- 0.8/40.1 MB 4.2 MB/s eta 0:00:10
   - -------------------------------------- 1.6/40.1 MB 4.0 MB/s eta 0:00:10
   -- ------------------------------------- 2.4/40.1 MB 3.9 MB/s eta 0:00:10
   --- ------------------------------------ 3.1/40.1 MB 3.9 MB/s eta 0:00:10
   --- ------------------------------------ 3.9/40.1 MB 3.9 MB/s eta 0:00:10
   ---- ----------------------------------- 5.0/40.1 MB 3.9 MB/s eta 0:00:10
   ----- ---------------------------------- 5.8/40.1 MB 3.8 MB/s eta 0:00:09
   ------ --------------------------------- 6.3/40.1 MB 3.7 MB/s eta 0:00:10
   ------- -------------------------------- 7.1/40.1 MB 3.7 MB/s eta 0:00:09
   ------- -------------------------------- 7.3/40.1 MB 3.6 MB/s eta 0:00:10
   -------- ------------------------------- 8.1/40.1 MB 3.5 MB/s eta 0:00:10
   -------- ------------------------------- 8.7/40.1 MB 3.4 MB/s eta 0:00:10
   ---


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import albumentations as A
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models

### Normalization

### augmantion